In [4]:
####   sensitive - means the parameter can effect the accuracy


import tensorflow.keras as K
import tensorflow as tf

def preprocess_data(X, Y):
    X_p = K.applications.resnet50.preprocess_input(X)
    Y_p = K.utils.to_categorial(Y, 7)  # 7 is for 7 emotions
    return X_p, Y_p

if __name__ == "__main__":
    train_images, train_labels = preprocess_data(train_image, train_labels)
    valid_images, valid_labels = preprocess_data(valid_images, valid_labels)
    
    input_t = K.Input(shape=(32,32,3))  # sensitive
    res_model = K.applications.Resnet50(include_top = False, weights = "imagenet", input_tensor = input_t)
    
    for layers in res_model.layers[:143]:  # sensitive
        layer.trainable = False
        
    for i, layer in enumerate(res_model.layers):
        print(i, layer.name, "-", layer.trainable)
    
    image_size_resnets = (32,32)   #  sensitive, more likely to increase the dimension to get more data
    
    model = K.models.Sequential()
    model.add(K.layers.Lambda(lambda image: tf.image.resize(image, image_size_resnets)))
    model.add(res_model)
    model.add(K.layers.Flatten())
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(256, activation = 'relu'))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalisation())
    model.add(K.layers.Dense(128, activation = 'relu'))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalisation())
    model.add(K.layers(Dense(64, activation = 'relu')))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalisation())
    model.add(K.layers.Dense(7, activation = 'softmax'))
    
    
    check_point = K.callbacks.ModelChechpoint(filepath = "resnets_on_emoji.h5", monitor = "val_acc", mode = "max", save_best_only = True)
    
    model.compile(loss = 'sparse_categorial_crossentropy',
                 optimizer = K.optimizers.Adam(), metrics = ['accuracy'])
    
    history = model.fit(train_images, train_labels, batch_size = 32, epochs = 10, verbose = 1,
                       validation_data = (valid_images, valid_labels), callbacks = [check_point])
    
    model.summary()
    
    
K.learning_phase = K.backend.learning_phase
###  removable part...  to check whether it affects the accuracy or not
train_images, train_labels = preprocess_data(train_image, train_labels)
valid_images, valid_labels = preprocess_data(valid_images, valid_labels)
###################

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = hisory.history['loss']
val_loss = history.history['val_loss']


epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label = 'Training_acc')
plt.plot(epochs, val_acc, 'bo', label = 'Validation_acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label = "Training_loss")
plt.plot(epochs, val_loss, 'ro', label = "Validation_loss")
plt.tilte('Training and Validation loss')
plt.legend()

plt.show()


NameError: name 'train_image' is not defined